In [121]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [123]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 10:00:00+00:00


In [124]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [125]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 04:15:58,360 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:15:58,366 INFO: Initializing external client
2025-03-05 04:15:58,367 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 04:15:58,969 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710


In [126]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.15s) 


In [127]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,13,31.0,2025-03-05 08:00:00+00:00
1,41,15.0,2025-03-05 08:00:00+00:00
2,53,0.0,2025-03-05 08:00:00+00:00
3,81,0.0,2025-03-05 08:00:00+00:00
4,93,0.0,2025-03-05 08:00:00+00:00
...,...,...,...
751,144,10.0,2025-03-05 10:00:00+00:00
752,178,0.0,2025-03-05 10:00:00+00:00
753,29,0.0,2025-03-05 10:00:00+00:00
754,43,88.0,2025-03-05 10:00:00+00:00


In [128]:
#a.info()

In [129]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-05 04:16:02,953 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:02,967 INFO: Initializing external client
2025-03-05 04:16:02,967 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:03,430 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Current UTC time: 2025-03-05 09:16:02.953568+00:00
Next hour: 2025-03-05 10:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 10:00
Found 252 records


In [130]:
now = datetime.now(timezone.utc)

In [131]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
504,45,2.0,2025-03-05 10:00:00+00:00
505,203,0.0,2025-03-05 10:00:00+00:00
506,231,26.0,2025-03-05 10:00:00+00:00
507,132,199.0,2025-03-05 10:00:00+00:00
508,260,1.0,2025-03-05 10:00:00+00:00
...,...,...,...
751,144,10.0,2025-03-05 10:00:00+00:00
752,178,0.0,2025-03-05 10:00:00+00:00
753,29,0.0,2025-03-05 10:00:00+00:00
754,43,88.0,2025-03-05 10:00:00+00:00


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  756 non-null    int32                  
 1   predicted_demand    756 non-null    float64                
 2   pickup_hour         756 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.9 KB


In [133]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [134]:
dt = current_date.ceil('h')


In [135]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 04:16:04,346 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:04,352 INFO: Initializing external client
2025-03-05 04:16:04,352 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:04,969 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.46s) 


In [136]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.70s) 


,pickup_location_id,predicted_demand,pickup_hour


In [137]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,13,31.0,2025-03-05 08:00:00+00:00
1,41,15.0,2025-03-05 08:00:00+00:00
2,53,0.0,2025-03-05 08:00:00+00:00
3,81,0.0,2025-03-05 08:00:00+00:00
4,93,0.0,2025-03-05 08:00:00+00:00
...,...,...,...
751,144,10.0,2025-03-05 10:00:00+00:00
752,178,0.0,2025-03-05 10:00:00+00:00
753,29,0.0,2025-03-05 10:00:00+00:00
754,43,88.0,2025-03-05 10:00:00+00:00


In [138]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [139]:
predictions = fetch_next_hour_predictions()

2025-03-05 04:16:23,268 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:23,279 INFO: Initializing external client
2025-03-05 04:16:23,279 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:23,781 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Current UTC time: 2025-03-05 09:16:23.268794+00:00
Next hour: 2025-03-05 10:00:00+00:00
Found 252 records


In [140]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-05 04:16:24,541 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:24,546 INFO: Initializing external client
2025-03-05 04:16:24,546 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:25,099 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710


In [141]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


In [142]:
df["pickup_hour"].max()

Timestamp('2025-03-05 10:00:00+0000', tz='Etc/UTC')

In [143]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 10:00:00+00:00


In [144]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
504,45,2.0,2025-03-05 10:00:00+00:00
505,203,0.0,2025-03-05 10:00:00+00:00
506,231,26.0,2025-03-05 10:00:00+00:00
507,132,199.0,2025-03-05 10:00:00+00:00
508,260,1.0,2025-03-05 10:00:00+00:00
...,...,...,...
751,144,10.0,2025-03-05 10:00:00+00:00
752,178,0.0,2025-03-05 10:00:00+00:00
753,29,0.0,2025-03-05 10:00:00+00:00
754,43,88.0,2025-03-05 10:00:00+00:00
